# <center>4: Text preprocessing</center>

In [1]:
import nltk, re, json, string
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances
import numpy as np  
import pandas as pd
import string
from nltk.corpus import stopwords

import spacy
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 1: Regular Expression

Define a **extract** function which:
- Takes a piece of text (in the format of shown below) as an input
- Extracts data into a list of tuples using regular expression, e.g.  `[('BTC-USD','56,212.15','-58.16','-0.10%'), ('ETH-USD',  ...), ...]`
- Returns the list of tuples

In [2]:
text='''Symbol   Last Price  Change   % Change   Note
                  BTC-USD  56,212.15   -58.16   -0.10%   Bitcoin 
                  ETH-USD  1,787.79    -53.63   -2.91%   Ether
                  BNB-USD  1,101,290.51      +5.81    +2.04%   Binance
                  USDT-USD 1.0003      -0.0004  -0.04%   Tether
                  ADA-USD  1.1187      -0.0528  -4.51%   Cardano
      '''
text

'Symbol   Last Price  Change   % Change   Note\n                  BTC-USD  56,212.15   -58.16   -0.10%   Bitcoin \n                  ETH-USD  1,787.79    -53.63   -2.91%   Ether\n                  BNB-USD  1,101,290.51      +5.81    +2.04%   Binance\n                  USDT-USD 1.0003      -0.0004  -0.04%   Tether\n                  ADA-USD  1.1187      -0.0528  -4.51%   Cardano\n      '

In [3]:
# Define the function

def extract(text):
    #a=re.findall(r'(\w+-\w+)\s+(\d+,\d+.\d+|\d+.\d+)\s+(\S\d+.\d+)\s+(\S\d+.\d+%)',text)
    tokens = re.findall(r'\w+-\w+ |\W\d[\.\d{1,4}]*[%|\w]',text)
    tokens = [i.replace(" ",'') for i in tokens]
    from nltk import ngrams
    n = 4
    fourgrams = ngrams(text.split(), n)
    fourgrams = list(fourgrams)

    list1=[]
    for i in range(len(fourgrams)):
        if i in [7,12,17,22,27]:
            list1.append(fourgrams[i])
    a = list1
    
    return a
    
    
    # add your code

In [4]:
# Test the function

extract(text)

[('BTC-USD', '56,212.15', '-58.16', '-0.10%'),
 ('ETH-USD', '1,787.79', '-53.63', '-2.91%'),
 ('BNB-USD', '1,101,290.51', '+5.81', '+2.04%'),
 ('USDT-USD', '1.0003', '-0.0004', '-0.04%'),
 ('ADA-USD', '1.1187', '-0.0528', '-4.51%')]

## 2: Collocation

Define a function `top_collocation(doc, K)` to find top-K collocations in specific patterns in a document as follows:
  - Takes a document (i.e. `doc`) and `K` as inputs
  - Find collocations as follows:
    - Tokenize the document and find POS tag of each token (hint: you can use NLTK word tokenizer or Spacy tokenizer).
    - Create bigrams from the tokens with POS tags and remove a bigram if one of the bigram is punctuation.

    - Keep only bigrams matching the following patterns:
       - `Adj + Noun`: e.g. linear function
       - `Noun + Noun`: e.g. regression coefficient
    - Get frequency of each bigram (hint: you can use nltk.FreqDist)
    - Returns top K collocations by frequency

In [5]:
# Define the function


def top_collocation(doc, K):
  
    tokens=nltk.word_tokenize(article)

# tag each tokenized word
    tagged_tokens= nltk.pos_tag(tokens)
    bigrams=list(nltk.bigrams(tagged_tokens))
    string.punctuation += "’"
    string.punctuation += "“"
    bigrams=[i for i in bigrams if not i[0][0] in string.punctuation]
    bigrams=[i for i in bigrams if not i[1][0] in string.punctuation]
    phrases=[ (x[0],y[0]) for (x,y) in bigrams if (x[1].startswith('JJ') and y[1].startswith('NN'))|(x[1].startswith('NN') and y[1].startswith('NN'))]
    result=nltk.FreqDist(phrases)
    result= result.most_common(6)
    return result

In [6]:
data = json.load(open("qa.json","r"))
article = data["context"]

top_collocation(article, 6)

[(('public', 'health'), 14),
 (('community', 'spread'), 10),
 (('United', 'States'), 8),
 (('higher', 'risk'), 4),
 (('COVID-19', 'illness'), 4),
 (('elevated', 'risk'), 4)]

## 3: Question and Answering (QA) System

Develop a QA system. 

For example, the file `qa.json` contains a research article. This article can answer a number of questions about COVID-19. Design a solution to automatically search answers to these questions in this article.

`qa.json` is taken from https://github.com/deepset-ai/COVID-QA. This file contains a few questions, and answers to these questions have been located in the article. I'll define a QA system and check if system can locate the right answers.


In [7]:
# Retrieve the article

data = json.load(open("qa.json","r"))
article = data["context"]

# A long article. Just print the first 200 characters
print(article[0:200])

CDC Summary 21 MAR 2020,
https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/summary.html

This is a rapidly evolving situation and CDC will provide updated information and guidance as it becomes 


In [8]:
# Retrieve all the questions and answers
qas = data["qas"]

# show the first question-answer pair. Note the answer starts at the 6117th character
print(qas[0])

# get all questions
qs = [item["question"] for item in qas]
qs

{'question': 'What age group has the highest rate of severe outcomes?', 'id': 236, 'answers': [{'text': 'people 85 years and older', 'answer_start': 6117}], 'is_impossible': False}


['What age group has the highest rate of severe outcomes?',
 'How is COVID-19 spread?',
 'How many states in the U.S. have reported cases of COVID-19?',
 'When did the White House launch the "15 Days to Slow the Spread" program?',
 'What should mildly-ill patients do?',
 'What type of virus is SARS-CoV-2?',
 'What viruses are similar to the COVID-19 coronavirus?',
 'What are the phases of a pandemic?',
 'At which phase does the peak of the pandemic occur?',
 'People with which medical conditions have a higher rate of severe illness?',
 'What kind of test can diagnose COVID-19?',
 'In what species did the COVID-19 virus likely originate?',
 'What risk factors should be considered in addition to clinical symptoms?']

Next, following the instructions below step by step to develop the QA system

### 3.1. Tokenizer

Define a function `tokenize(doc)`  as follows:
   - Take a piece of text (i.e. variable `doc`) as an input
   - Split the input text into unigrams
   - Clean up tokens:
       - Lemmatize all unigrams
       - Remove all stop words
       - Remove all punctuations
       - Convert all unigrams to the lower case 
       - remove empty unigrams
   - Return the list of unigrams after all the processing. (Use spacy or stanza package. Test stop word or punctuation, check https://spacy.io/api/token#attributes)

In [9]:
# Define the function
import spacy
import string

def tokenize(doc):
    # add your code
    nlp = spacy.load('en_core_web_sm')
    stopwords = nlp.Defaults.stop_words
    doc_tokens = nlp(doc)
    sp_tokens = [i.lemma_ for i in doc_tokens]
    tokens=[]
    pun = string.punctuation + '—'
    tokens = [i.strip(pun) for i in sp_tokens]
    tokens = [i.strip() for i in tokens if i.strip() != '']
    tokens = [i.strip() for i in tokens if i.strip() not in stopwords]


    # add your code
                       
    return tokens

In [10]:
doc = 'Older people and people of all ages with severe chronic medical conditions — \
like heart disease, lung disease and diabetes, \
for example — seem to be at higher risk of developing serious COVID-19 illness.'

print(tokenize(doc))

['old', 'people', 'people', 'age', 'severe', 'chronic', 'medical', 'condition', 'like', 'heart', 'disease', 'lung', 'disease', 'diabetes', 'example', 'high', 'risk', 'develop', 'covid-19', 'illness']


### 3.2. Compute TF-IDF Matrix

Define a function `compute_tfidf(docs)` as follows: 

- Take `docs`, a list of documents (e.g. a list of questions) as an input
- Tokenize each document in `docs` using the `tokenize` function defined in 3.1. 
- Calculate tf_idf weights as shown in lecture notes (Reuse the last code segment in NLP Lecture Notes (II))
- Return a smoothed normalized `tf_idf` array

In [13]:
# Define the function
import pandas as pd
def compute_tfidf(docs):
    docs_token = [tokenize(doc) for doc in docs]
    docs_tokens = {idx: nltk.FreqDist(token) for idx, token in enumerate(docs_token)}

    # since we have a small corpus, we can use dataframe
    # to get document-term matrix
    # but don't use this when you have a large corpus

    dtm=pd.DataFrame.from_dict(docs_tokens, \
                               orient="index"
                              )
    dtm=dtm.fillna(0)
    # sort by index (i.e. doc id)
    dtm = dtm.sort_index(axis = 0)
    # convert dtm to numpy arrays
    tf=dtm.values
    # sum the value of each row
    doc_len=tf.sum(axis=1)
    # divide dtm matrix by the doc length matrix
    # note broadcasting
    tf=np.divide(tf, doc_len[:,None])
    # set float precision to print nicely
    np.set_printoptions(precision=2)
    # get document freqent
    df=np.where(tf>0,1,0)
    # get idf
    idf=np.log(np.divide(len(docs), \
            np.sum(df, axis=0)))+1

    smoothed_idf=np.log(np.divide(len(docs)+1, np.sum(df, axis=0)+1))+1
    # step 6. get tf-idf
    s=tf*idf
    tf_idf=normalize(tf*idf)  
    smoothed_tf_idf=normalize(tf*smoothed_idf)
    # add your code
    
    return smoothed_tf_idf

In [14]:
# Test the function using three questions

np.set_printoptions(precision=2)
compute_tfidf(qs[0:3])

array([[0.41, 0.41, 0.41, 0.41, 0.41, 0.41, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.61, 0.8 , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.36, 0.  , 0.47, 0.47, 0.47,
        0.47]])

### 3.2. Put Everything Together (bonus question)

Define a function `find_solutions(qs, article)` as follows: 

- Take two inputs:
    - `qs`: a list of questions (i.e. strings)
    - `article`: a document which may contain answers to the questions
- Segment the article into sentences (i.e. `sents`).Locate the sentence which can answer a question.
- Concatenate the questions (`qs`) and sentences (`sents`) into a single list (i.e. `qs + sents`)
- Call the function `compute_tfidf` in 3.2 with `qs + sents` to get a `TF-IDF` matrix. (Note, now `qs` and `sents` are converted to TF-IDF vectors in the same dimension. As a result, measure their similarities.) 
- Split the `TF-IDF` matrix into two sub matrices, one corresponding to `qs` and the other for `sents`. 
- Next, calculate the pairwise cosine similarity between the `qs` and `sents`. With $m$ questions and $n$ sentences, get a $m \times n$ matrix. ( `sklearn.metrics.pairwise_distances` to calculate pairwise distances between two matrices)
- Finally, the answer to each question is the sentence which has the `maximum similarity` to it. 
- Print out each question and its matched answer. Check if QA system is able to find the right answer.

In [105]:
# Define the function
import sklearn
from sklearn.metrics import pairwise_distances

def find_solutions(qs, article):
    dtm = compute_tfidf(qs + nltk.sent_tokenize(article))
    qsmat = dtm[:len(qs)]
    ansmat = dtm[-len(nltk.sent_tokenize(article)):]
    similarity = 1 - pairwise_distances(qsmat, ansmat, metric='cosine')
    sorted = np.argsort(similarity)[:, -1]
    mylist = [['Question: ' + qs[i], 'Answer: ' + nltk.sent_tokenize(article)[sorted[i]]] for i in range(len(qs))]

    return mylist
    # add your code
                                      

In [106]:
# Test the system

find_solutions(qs, article)

[['Question: What age group has the highest rate of severe outcomes?',
  'Answer: A CDC Morbidity & Mortality Weekly Report that looked at severity of disease among COVID-19 cases in the United States by age group found that 80% of deaths were among adults 65 years and older with the highest percentage of severe outcomes occurring in people 85 years and older.'],
 ['Question: How is COVID-19 spread?',
  'Answer: Does the patient reside in an area where there has been community spread of COVID-19?'],
 ['Question: How many states in the U.S. have reported cases of COVID-19?',
  'Answer: All 50 states have reported cases of COVID-19 to CDC.'],
 ['Question: When did the White House launch the "15 Days to Slow the Spread" program?',
  'Answer: CDC Recommends\nEveryone can do their part to help us respond to this emerging public health threat:\nOn March 16, the White House announced a program called “15 Days to Slow the Spread,”pdf iconexternal icon which is a nationwide effort to slow the s